In [ ]:
import numpy as np
import scipy as sp
from scipy import io
import cv2 as cv 
import matplotlib.pyplot as plt
import pylab
import os # for reading all files in a folder
pylab.rcParams['figure.figsize'] = (3, 2.5)

Student Participant number: 05112003 
(Please note that this isn't the one attributed to me by UCL, but should be a unique string, as suggested by Gabriel Brostow's email)

# Part H: Tracking and Homographies

In this part we use Practical 7c to track the positions of the four corners of the square and project a cube into the images. 

TO DO: QUESTIONS TO THINK ABOUT...

- Do the results look realistic? 
- If not then what factors do you think might be causing this


TO DO: your routines for computing a homography and extracting a valid rotation and translation go in the code below. Tips:
- you may define functions for T and H matrices respectively.
- you may need to turn the points into homogeneous form before any other computation. 
- you may need to solve a linear system in Ah = 0 form. Write your own routines or using the builtin function 'svd'. 
- you may apply the direct linear transform (DLT) algorithm to recover the best homography H.
- you may explain what & why you did in the report.


In [ ]:
# TO DO: Copy and paste the function HW2_Practical7c in here. 
def computeLikelihood(image, template):
    #opencv's available methods - experiment with these
    #careful what range the output is!
    methods = [cv.TM_CCOEFF, cv.TM_CCOEFF_NORMED, cv.TM_CCORR,
            cv.TM_CCORR_NORMED, cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]
    
    likelihood = cv.matchTemplate(image[:,:,2], template, methods[0])
    # (You can also try converting the image to greyscale instead of using the third channel as above with 
    # cv.cvtColor(image, cv.COLOR_BGR2GRAY))
    
    #we can pad to make this the size of the input image (for easier indexing)   
    pad_first = int(template.shape[0])
    pad_second = int(template.shape[1])
    pad_amounts = ((0, pad_first-1), (0, pad_second-1))
    likelihood = np.pad(likelihood, pad_amounts, 'constant')
    likelihood[likelihood<0] = 0 # to avoid negative weights 
    
    # apply a 10x10 averaging filter for stability. You can experiment with different sizes. 
    kernel = np.ones((10,10),np.float32)/100
    smoothed = cv.filter2D(likelihood,-1,kernel) 
    return smoothed 

def HW2_Practical7c(corner):
    template = sp.io.loadmat(corner+'.mat')['pixelsTemplate']
    #let's show the template
    print('We are matching this template with shape: ', template.shape)
    plt.imshow(template)
    plt.show()

    # Load all images in folder
    images = []
    iFrame = 0
    folder = 'Pattern01/'
    lst = os.listdir(folder)
    lst.sort()

    for frameNum in lst:
        images.append(cv.imread(folder+frameNum))
        iFrame += 1
    # plot first image 
    plt.imshow(images[0])
    plt.show()

    imgHeight, imgWidth, colors = images[0].shape
    numParticles = 2000;
    weight_of_samples = np.ones((numParticles,1))

    # TO DO: normalize the weights (may be trivial this time) [done]
    weight_of_samples = weight_of_samples / np.sum(weight_of_samples)

    # Initialize which samples from "last time" we want to propagate: all of
    # them!:
    samples_to_propagate = range(0, numParticles)

    # ============================
    # NOT A TO DO: You don't need to change the code below, but eventually you may
    # want to vary the number of Dims (compare for example to lab 9b) 
    numDims_w = 2;
    # Here we randomly initialize some particles throughout the space of w:
    particles_old = np.random.rand(numParticles, numDims_w)
    particles_old[:,0] = particles_old[:,0] * imgHeight
    particles_old[:,1] = particles_old[:,1] * imgWidth
    # ============================

    #Initialize a temporary array r to store the per-frame MAP estimate of w. This is what we'll return in the end.
    r = np.zeros((iFrame, numDims_w));

    for iTime in range(iFrame):
        print('Processing Frame', iTime)
        # TO DO: compute the cumulative sume of the weights. [done] 
        cum_hist_of_weights = np.cumsum(weight_of_samples)
        #print(weight_of_samples)

        # ==============================================================
        # Resample the old distribution at time t-1, and select samples, favoring
        # those that had a higher posterior probability.
        # ==============================================================
        samples_to_propagate = np.zeros(numParticles, dtype=np.int32)

        # Pick random thresholds in the cumulative probability's range [0,1]:
        some_threshes = np.random.rand(numParticles)

        # For each random threshold, find which sample in the ordered set is
        # the first one to push the cumulative probability above that
        # threshold, e.g. if the cumulative histogram goes from 0.23 to 0.26
        # between the 17th and 18th samples in the old distribution, and the
        # threshold is 0.234, then we'll want to propagate the 18th sample's w
        # (i.e. particle #18).

        for sampNum in range(numParticles): 
            thresh = some_threshes[sampNum]
            for index in range (numParticles):
                if cum_hist_of_weights[index] > thresh:
                    break
            samples_to_propagate[sampNum] = index

        # Note: it's ok if some of the old particles get picked repeatedly, while
        # others don't get picked at all.

        # =================================================
        # Visualize if you want
        # =================================================
        #plt.title('Cumulative histogram of probabilities for sorted list of particles')
        #plt.plot(np.zeros(numParticles), some_threshes,'b.')
        #plt.plot(range(0, numParticles), cum_hist_of_weights, 'rx-')
        #which_sample_ids = np.unique(samples_to_propagate)
        #how_many_of_each = np.bincount(np.ravel(samples_to_propagate))
        #for k in range(len(which_sample_ids)):
        #    plt.plot(which_sample_ids[k], 0, 'bo-', markersize = 3 * how_many_of_each[k], markerfacecolor='white')
        #plt.xlabel('Indeces of all available samples, with larger blue circles for frequently re-sampled particles\n(Iteration %01d)' % iTime)
        #plt.ylabel('Cumulative probability');
        #plt.show()
        # =================================================
        # =================================================

        # Predict where the particles we sampled from the old distribution of 
        # state-space will go in the next time-step. This means we have to apply 
        # the motion model to each old sample.
        particles_new = np.zeros_like(particles_old)
        for particleNum in range(numParticles):
            # TO DO: Incorporate some noise, e.g. Gaussian noise with std 20,
            # into the current location (particles_old), to give a Brownian
            # motion model.
            old_idx = samples_to_propagate[particleNum]
            noise = np.random.normal(0, 20, size=(2,))
            particles_new[particleNum, :] = particles_old[old_idx, :] + noise
            
        # TO DO: Not initially, but change the motion model above to have
        # different degrees of freedom, and optionally completely different
        # motion models. See Extra Credit for more instructions.

        #calculate likelihood function
        likelihood = computeLikelihood(images[iTime], template)

        #plot results
        f, axarr = plt.subplots(1, 2)
        axarr[0].imshow(images[iTime])
        axarr[0].set_title('Particles')
        # now draw the particles onto the image
        axarr[0].plot(particles_new[:,1]+template.shape[1]/2, particles_new[:,0]+template.shape[0]/2, 'rx')

        #plot the likelihood
        axarr[1].imshow(likelihood)
        axarr[1].set_title('Likelihood')

        # From here we incorporate the data for the new state (time t):
        # The new particles accompanying predicted locations in state-space
        # for time t, are missing their weights: how well does each particle
        # explain the observations x_t?
        for particleNum in range(numParticles):

            # Convert the particle from state-space w to measurement-space x:
            # Note: that step is trivial here since both are in 2D space of image
            # coordinates

            # Within the loop, we evaluate the likelihood of each particle:
            particle = particles_new[particleNum, :]
            # Check that the predicted location is a place we can really evaluate
            # the likelihood.
            inFrame = particle[0] >= 0.0 and  particle[0] <= imgHeight and particle[1] >= 0.0 and particle[1] <= imgWidth
            if inFrame:
                minX = particle[1]
                minY = particle[0]

                weight_of_samples[particleNum] = likelihood[int(minY), int(minX)]

            else:
                weight_of_samples[particleNum] = 0.0

        # TO DO: normalize the weights [done]                         
        sum_weights = np.sum(weight_of_samples)
        weight_of_samples = weight_of_samples / sum_weights
        
        # find the location of the particle with highest weight
        indices = np.argsort(weight_of_samples,0)
        bestScoringParticles = particles_new[np.squeeze(indices[-15:]), :]
        plt.plot(bestScoringParticles[-1:,1], bestScoringParticles[-1:,0], 'rx')
        # Return the MAP of middle position. Add template.shape/2 because matchTemplate finds the position of the upper left corner 
        # of the template. We want to plot the centre of the template. 
        r[iTime,:] = bestScoringParticles[-1,1]+template.shape[1]/2,bestScoringParticles[-1,0]+template.shape[0]/2
        print(r[iTime,:])   
        plt.show()

        #print the original image and the position of the tracked corner.
        plt.imshow(images[iTime])
        plt.plot(r[iTime,0],r[iTime,1],'rx')
        plt.show()
        # Now we're done updating the state for time t. 
        # For Condensation, just clean up and prepare for the next round of 
        # predictions and measurements:
        particles_old = particles_new

    return r

In [ ]:
# instead of running the code here, you can also save the output of each function in a numpy array in HW2_Practical7c 
# and load it here. This could be handy if you need different hyperparameters for each corner.
LLs = HW2_Practical7c( 'll' )
LRs = HW2_Practical7c( 'lr' )
ULs = HW2_Practical7c( 'ul' )
URs = HW2_Practical7c( 'ur' )

### Figure dicussion: 
This is the same thing as in the HW2_Practical7c, where the figures above show the evolution of a particle filter tracking each corner of a black square in turn across frames. Initially, the particles are scattered randomly. At the start, particles can be found in other areas of high probability, but in the end they all stabilise tightly around the corner.


In [ ]:
# Load all images in folder
images = []
nFrame = 0
folder = 'Pattern01/'
lst = os.listdir(folder)
lst.sort()

for frameNum in lst:
    images.append(cv.imread(folder+frameNum))
    nFrame += 1
# plot first image 
plt.imshow(images[0])
plt.show()


# Coordinates of the known target object (a dark square on a plane) in 3D:
XCart = np.array([[-50, -50,  50,  50],
          [50, -50, -50,  50],
            [0, 0, 0, 0]])

# These are some approximate intrinsics for this footage.
K = np.array([[640, 0, 320],
          [0, 512, 256],
            [0, 0, 1]])

# Define 3D points of wireframe object.
XWireFrameCart = np.array([[-50, -50,  50,  50, -50, -50,  50,  50],
          [50, -50, -50,  50, 50, -50, -50,  50],
            [0, 0, 0, 0, -100, -100, -100, -100, ]])

### Figure dicussion: 
The figure above is a plot of the first image contained in the data.

In [ ]:
# Copy pasting functions to calculate the extrinsic matrix
def solveAXEqualsZero(A):
    # TODO: Write this routine - it should solve Ah = 0. You can do this using SVD. Consult your notes! 
    # Hint: SVD will be involved. 
  
    # We compute the SVD of A
    U, S, Vt = np.linalg.svd(A)
    
    # The vector h is the last column of V corresponding to the smallest singular value
    h = Vt[-1, :]
    
    # We normalize h if we want the last element to be 1.
    if abs(h[-1]) > 1e-12:
        h = h / h[-1]
    return h

def calcBestHomography(pts1Cart, pts2Cart):
    # This function should apply the direct linear transform (DLT) algorithm to calculate the best 
    # homography that maps the cartesian points in pts1Cart to their corresonding matching cartesian poitns 
    # in pts2Cart.
    
    # This function calls solveAXEqualsZero. Make sure you are wary of how to reshape h into a 3 by 3 matrix. 

    n_points = pts1Cart.shape[1]
    
    # TODO: replace this:
    H = np.identity(3)

    # TODO: 
    # First convert points into homogeneous representation
    # Hint: we've done this before  in the skeleton code we provide.
    pts1Hom = np.vstack((pts1Cart, np.ones((1, n_points))))
    pts2Hom = np.vstack((pts2Cart, np.ones((1, n_points))))

    # TODO: 
    # Then construct the matrix A, size (n_points * 2, 9)
    # Consult the notes!
    A = np.zeros((2 * n_points, 9))

    for i in range(n_points):
        x,  y  = pts1Hom[0, i],  pts1Hom[1, i]
        x_p, y_p = pts2Hom[0, i], pts2Hom[1, i]
        A[2*i, :]   = [0, 0, 0, -x, -y, -1,  y_p*x,  y_p*y,  y_p]
        A[2*i+1, :] = [x, y, 1,  0,  0,  0, -x_p*x, -x_p*y, -x_p]

    # TODO: 
    # Solve Ah = 0 using solveAXEqualsZero and get h.
    h = solveAXEqualsZero(A)

    # TODO: 
    # Reshape h into the matrix H, values of h go first into rows of H
    H = h.reshape(3, 3)
    
    return H

def projectiveCamera(K,T,XCart):
    ##TODO
    # The goal of this function is to project points in XCart through projective camera
    # defined by intrinsic matrix K and extrinsic matrix T. In essence, this function takes a set of points 
    # in 3D world space, XCart, and projects them into camera image space by applying the extrinsic matrix T 
    # and then applying the intrinsic matrix K.
    # 
    # There are three steps.
    # 1) Move from world space to camera space. 
    #            camera space points = extrinsics T * world space points 
    #
    # 2) Applying the intrinsics matrix to the camera space points after normalizing
    #           homogeneous image space points = K * normalized camera space points
    # 
    # 3) Move to image space cartesian points from image space homogeneous points, involves a 
    # normalization using the third row.
    
    # TODO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    n = XCart.shape[1]
    XHom = np.vstack((XCart, np.ones((1, n))))

    # TODO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    xCamHom = T @ XHom 

    # TODO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    xCamCart = xCamHom[:3, :] / xCamHom[3, :]

    # TODO: Move points to image coordinates xImHom by applying intrinsic matrix
    xImHom = K @ xCamCart

    # TODO: Convert points back to Cartesian coordinates xImCart
    XImCart = xImHom[:2, :] / xImHom[2, :]

    return XImCart

def estimatePlanePose(XImCart,XCart,K):
    # The goal of this function is to estimate the pose of a plane relative to camera (extrinsic matrix)
    # given points in image space xImCart, points in 3D world space XCart, and an intrinsics matrix K.

    # TODO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    n = XImCart.shape[1]
    XImHom = np.vstack((XImCart, np.ones((1, n))))
    
    # TODO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    K_inv = np.linalg.inv(K)
    XCamHom = K_inv @ XImHom
    xCamNorm = XCamHom[:2, :] / XCamHom[2, :]
    
    # TODO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    XCart2D = XCart[:2, :]
    H_est = calcBestHomography(XCart2D, xCamNorm)
          
    # TODO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD. NOTE: You do not need to transpose v from linalg.svd    
    h1 = H_est[:, 0]
    h2 = H_est[:, 1]
    h3 = H_est[:, 2]
    Q = np.column_stack((h1, h2))
    U, S, Vt = np.linalg.svd(Q)
    R12 = U[:, :2] @ Vt
    r1 = R12[:, 0]
    r2 = R12[:, 1]

    # TODO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    r3 = np.cross(r1, r2)
    R = np.column_stack((r1, r2, r3))
        
    # TODO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    if np.linalg.det(R) < 0:
        R[:, 2] = -R[:, 2]
    
    # TODO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    norm_h1 = np.linalg.norm(h1)
    norm_h2 = np.linalg.norm(h2)
    k = (norm_h1 + norm_h2) / 2.0
    t = h3 / k
    
    # TODO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[2] < 0:
        R[:, 0] = -R[:, 0]
        R[:, 1] = -R[:, 1]
        t = -t
            
    # TODO: Assemble transformation into matrix form
    T = np.eye(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = t
    
    return T 



In [ ]:
# ================================================
for iFrame in range(nFrame):
    if iFrame % 10 == 0:
        xImCart = np.array([LLs[iFrame,:].T, ULs[iFrame,:].T, URs[iFrame,:].T, LRs[iFrame,:].T]).T

        # get a frame from footage 
        im = images[iFrame]

        # Draw image and 2d points
        plt.imshow(im)
        plt.scatter(x = xImCart[0,:], y = xImCart[1,:],c = 'r')
        plt.show()

        #TO DO: Use your routine to calculate TEst the extrinsic matrix relating the
        #plane position to the camera position.
        TEst = estimatePlanePose(xImCart, XCart, K);
    
        # TO DO: Draw a wire frame cube using data XWireFrameCart. You need to
        # 1) project the vertices of a 3D cube through the projective camera;
        # 2) draw lines betweeen the resulting 2d image points.
        # Note: CONDUCT YOUR CODE FOR DRAWING XWireFrameCart HERE

        XWireFrameCartProjected = projectiveCamera(K, TEst, XWireFrameCart)

        # We define edges of the cube:
        edges = np.array([
       [0,1], [1,2], [2,3], [3,0], 
       [4,5], [5,6], [6,7], [7,4], 
       [0,4], [1,5], [2,6], [3,7]   
        ])

        plt.imshow(im)
        plt.plot(xImCart[0, :], xImCart[1, :], 'r.')

        for e in edges:
           plt.plot(
                [XWireFrameCartProjected[0, e[0]], XWireFrameCartProjected[0, e[1]]],
              [XWireFrameCartProjected[1, e[0]], XWireFrameCartProjected[1, e[1]]],
                'g-'
            )    

        plt.axis('off')
        plt.show()


### Figure discussion:
Included above are every 10th frame from the data. This was done in the interest of storage space and to illustrate the result better. The figures contain red dots, marking out the corners of the black square. These dots are used to place a 3D cube in the world space. Most frames give good results, as they give convicing renditions of the cube. Some on the other hand perform a lot worse, such as the last displayed image. The results look in general realistic, and when they don't, it could be an issue with the estimation linked to performing the Direct Linear Transformation (DLT). 

## Question 1: 
Consider this simplistic example, and mention at least two actions or changes we could make to improve the results

## Answer - Question 1: 


Below are some suggestions to improve the results: 

**Use a more realistic motion model:** Instead of applying pure Brownian noise with a standard deviation to update particle positions, we could incorporate velocity or acceleration states into the model. This would be more physically plausible and would help the tracker adapt to the movement patterns, which might make the dots allign better.

**Try other resampling techniques:** Instead of threshold-based resampling, we could use systematic (low-variance) or stratified resampling. These can help maintain diversity, as multinomial resampling often resamples only the highest weight particles and discards some informative ones. By preserving more medium weight particles,we could improve the model's performance in the face of movement for example.  

**Adding more features to track** For now we are only tracking the four corners of the black square, we could add more reference points to improve the results, such as the centre of the square of the middle of the vertices. This woul dbe more computationaly expensive but might improve how the 3D cube aligns on the black square. 